In [1]:
import pandas as pd

df = pd.read_csv("../data_selected/syncan/train_flags.csv")

In [2]:
df.columns

Index(['Label', 'Time', 'ID', 'Signal1', 'Signal2', 'Signal3', 'Signal4',
       'Signal1_Missing', 'Signal2_Missing', 'Signal3_Missing',
       'Signal4_Missing'],
      dtype='object')

In [3]:
import pandas as pd

import math

def car_hacking_sessionize(df, num_per_session=10):
    df.fillna(0, inplace=True)

    grouped = df.groupby(['ID'])

    # Get max number of rows in a group
    print("Number of rows in the DataFrame: ", len(df))
    print("Number of groups: ", grouped.ngroups)
    print("Max number of rows in a group: ", grouped.size().max())

    # Global session ID counter
    global_session_id = 0

    # Iterate over each group
    for group_key, group in grouped:
        # print("Group key: ", group_key)
        # Calculate the number of sessions in this group
        total_rows = len(group)
        sessions_in_group = math.ceil(total_rows / num_per_session)  # Ceiling division

        # Assign session IDs within this group
        for session_number in range(sessions_in_group):
            # Determine the row indices for this session
            session_start = session_number * num_per_session
            session_end = session_start + num_per_session
            # Assign the global session ID to these rows
            df.loc[group.index[session_start:session_end], 'session_id'] = global_session_id
            # Increment the global session ID
            global_session_id += 1
    
    # Check if number of sessions is correct
    assert global_session_id == df['session_id'].nunique() == df.groupby(['session_id', 'ID']).ngroups
    print("Number of sessions: ", global_session_id)

    return df

final_df = car_hacking_sessionize(pd.read_csv("../data_selected/syncan/train_flags.csv"))

Number of rows in the DataFrame:  1000000
Number of groups:  10
Max number of rows in a group:  140123
Number of sessions:  100005


In [4]:
final_df.to_csv("../data_selected/syncan/train_flags_sessionized.csv", index=False)